In [22]:
%pip install pyspark
%pip install pymongo
%pip install dnspython
%pip install numpy


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/cc/05/ef9fc04adda45d537619ea956bc33489f50a46badc949c4280d8309185ec/numpy-1.26.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 61.1/61.1 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 15.8/15.8 MB 16.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pyspark
import pymongo
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame


In [13]:
%pip install mongo-spark-connector

Note: you may need to restart the kernel to use updated packages.


In [14]:
from pyspark.sql import SparkSession


In [17]:
# Read the CSV file into a DataFrame
file_path = "./data/continuous dataset.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first 10 rows before filtering
print("Before filtering:")
df.show(10, truncate=False)

# Select only the datetime and nat_demand columns
df_filtered = df.select("datetime", "nat_demand")

# Show the first 10 rows after filtering
print("After filtering:")
df_filtered.show(10, truncate=False)



Before filtering:
+-------------------+----------+------------------+-----------+------------+------------------+------------------+-----------+------------+------------------+------------------+-----------+-----------+------------------+----------+-------+------+
|datetime           |nat_demand|T2M_toc           |QV2M_toc   |TQL_toc     |W2M_toc           |T2M_san           |QV2M_san   |TQL_san     |W2M_san           |T2M_dav           |QV2M_dav   |TQL_dav    |W2M_dav           |Holiday_ID|holiday|school|
+-------------------+----------+------------------+-----------+------------+------------------+------------------+-----------+------------+------------------+------------------+-----------+-----------+------------------+----------+-------+------+
|2015-01-03 01:00:00|970.345   |25.865258789062523|0.018576382|0.016174316 |21.85054581787518 |23.482446289062523|0.017271755|0.0018553734|10.328948729384228|22.662133789062523|0.016562222|0.09609985 |5.364147952093894 |0         |0      |0 

In [25]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import dayofweek

# Add time-based features
df_filtered = df_filtered.withColumn("day_of_week", dayofweek("datetime"))
df_filtered = df_filtered.withColumn("hour", hour("datetime"))

# Assemble features into a single vector
feature_columns = ["day_of_week", "hour", "nat_demand"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_features = assembler.transform(df_filtered)

# Perform K-means clustering
kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(df_features)

# Make predictions
predictions = model.transform(df_features)

# Show the resulting clusters
predictions.select("datetime", "day_of_week", "hour", "nat_demand", "prediction").show(20)

# You can also examine the cluster centers
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)


+-------------------+-----------+----+----------+----------+
|           datetime|day_of_week|hour|nat_demand|prediction|
+-------------------+-----------+----+----------+----------+
|2015-01-03 01:00:00|          7|   1|   970.345|         0|
|2015-01-03 02:00:00|          7|   2|  912.1755|         0|
|2015-01-03 03:00:00|          7|   3|  900.2688|         0|
|2015-01-03 04:00:00|          7|   4|  889.9538|         0|
|2015-01-03 05:00:00|          7|   5|  893.6865|         0|
|2015-01-03 06:00:00|          7|   6|  879.2323|         0|
|2015-01-03 07:00:00|          7|   7|  932.4876|         0|
|2015-01-03 08:00:00|          7|   8|  1048.972|         2|
|2015-01-03 09:00:00|          7|   9| 1167.9074|         4|
|2015-01-03 10:00:00|          7|  10| 1257.5069|         4|
|2015-01-03 11:00:00|          7|  11|  1254.583|         4|
|2015-01-03 12:00:00|          7|  12| 1216.9004|         4|
|2015-01-03 13:00:00|          7|  13| 1202.1556|         4|
|2015-01-03 14:00:00|   

In [28]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
# Group by cluster and calculate average and standard deviation
cluster_stats = predictions.groupBy("prediction").agg(
    F.mean("nat_demand").alias("avg_demand"),
    F.stddev("nat_demand").alias("std_demand"),
    F.min("nat_demand").alias("min_demand"),
    F.max("nat_demand").alias("max_demand")
).orderBy("avg_demand")

# Show cluster statistics
cluster_stats.show()

# Identify peak and low demand clusters
peak_cluster = cluster_stats.orderBy(F.desc("avg_demand")).first().prediction
low_cluster = cluster_stats.orderBy("avg_demand").first().prediction

# Extract times falling into peak and low demand clusters
peak_times = predictions.filter(predictions.prediction == peak_cluster).select("datetime", "nat_demand")
low_times = predictions.filter(predictions.prediction == low_cluster).select("datetime", "nat_demand")

# Show peak and low demand times
print("Peak Demand Times:")
peak_times.show()

print("Low Demand Times:")
low_times.show()



+----------+------------------+------------------+------------------+----------+
|prediction|        avg_demand|        std_demand|        min_demand|max_demand|
+----------+------------------+------------------+------------------+----------+
|         0| 948.6941679945553| 51.70354086833256| 85.19250000000002| 1019.9557|
|         2|1088.1731736523966|40.664910880681205|          1019.453| 1160.9232|
|         4| 1229.164280808098| 39.36214528566216|         1160.3637| 1297.6514|
|         1|1361.6491870616849| 39.92515129585863|1297.6509999999998| 1434.7894|
|         3|1504.7913107896306| 52.11418229897436|1434.7267000000002|  1754.882|
+----------+------------------+------------------+------------------+----------+

Peak Demand Times:
+-------------------+----------+
|           datetime|nat_demand|
+-------------------+----------+
|2015-01-06 13:00:00|   1437.47|
|2015-01-12 11:00:00|  1452.004|
|2015-01-12 12:00:00| 1439.2499|
|2015-01-12 13:00:00| 1468.9201|
|2015-01-12 14:00:00

In [31]:
from pyspark.sql.window import Window

# Calculate Z-scores for demand in each cluster
z_score_window = Window.partitionBy('prediction')
predictions_with_z_score = predictions.withColumn(
    'avg_demand',
    F.avg('nat_demand').over(z_score_window)
).withColumn(
    'std_demand',
    F.stddev('nat_demand').over(z_score_window)
).withColumn(
    'z_score',
    (F.col('nat_demand') - F.col('avg_demand')) / F.col('std_demand')
)

# Show Z-scores
print("Z-Scores:")
predictions_with_z_score.select('datetime', 'nat_demand', 'prediction', 'z_score').show()

# Identify times when demand is unusually high or low
anomalous_high = predictions_with_z_score.filter(F.col('z_score') > 3.5).select('datetime', 'nat_demand', 'z_score')
anomalous_low = predictions_with_z_score.filter(F.col('z_score') < -3.5).select('datetime', 'nat_demand', 'z_score')

# Show anomalous times
print("Anomalous High Demand Times:")
anomalous_high.show()

print("Anomalous Low Demand Times:")
anomalous_low.show()


Z-Scores:
+-------------------+----------+----------+--------------------+
|           datetime|nat_demand|prediction|             z_score|
+-------------------+----------+----------+--------------------+
|2015-01-03 01:00:00|   970.345|         0|  0.4187495022938207|
|2015-01-03 02:00:00|  912.1755|         0| -0.7063088403859116|
|2015-01-03 03:00:00|  900.2688|         0| -0.9365967433038458|
|2015-01-03 04:00:00|  889.9538|         0| -1.1360995206139741|
|2015-01-03 05:00:00|  893.6865|         0| -1.0639052388044232|
|2015-01-03 06:00:00|  879.2323|         0| -1.3434644287022461|
|2015-01-03 07:00:00|  932.4876|         0|-0.31345180083176405|
|2015-01-03 23:00:00|   999.634|         0|   0.985229080057884|
|2015-01-04 00:00:00|  968.0526|         0|  0.3744121133742139|
|2015-01-04 01:00:00|  944.0556|         0|-0.08971470651052814|
|2015-01-04 02:00:00|  928.7193|         0|  -0.386334623491751|
|2015-01-04 03:00:00|  909.5566|         0| -0.7569610772736967|
|2015-01-04 04:

In [32]:
# Count the occurrences of each unique value in the 'prediction' column
cluster_distribution = predictions.groupBy("prediction").count().orderBy("prediction")

# Show the distribution
cluster_distribution.show()


+----------+-----+
|prediction|count|
+----------+-----+
|         0|11990|
|         1| 7798|
|         2|11401|
|         3| 6214|
|         4|10645|
+----------+-----+



In [37]:
# Extract the month, year, and date from the datetime column for each DataFrame
from pyspark.sql.functions import to_date
anomalous_high = anomalous_high.withColumn('date', to_date('datetime'))
peak_times = peak_times.withColumn('date', to_date('datetime'))
low_times = low_times.withColumn('date', to_date('datetime'))

# Find the date with the most anomalous demand times
most_anomalous_date = anomalous_high.groupBy('date').count().orderBy(F.desc('count')).first().date

# Find the date with the highest load
highest_load_date = peak_times.groupBy('date').count().orderBy(F.desc('count')).first().date

# Find the date with the lowest load
lowest_load_date = low_times.groupBy('date').count().orderBy('count').first().date

# Display the results
print(f"The date with the most anomalous demand times is: {most_anomalous_date}")
print(f"The date with the highest load is: {highest_load_date}")
print(f"The date with the lowest load is: {lowest_load_date}")


The date with the most anomalous demand times is: 2020-02-28
The date with the highest load is: 2020-01-20
The date with the lowest load is: 2015-06-27


In [38]:
# Find the top 3 dates with the most anomalous demand times
top_anomalous_dates = anomalous_high.groupBy('date').count().orderBy(F.desc('count')).limit(3).collect()

# Find the top 3 dates with the highest load
top_highest_load_dates = peak_times.groupBy('date').count().orderBy(F.desc('count')).limit(3).collect()

# Find the top 3 dates with the lowest load
top_lowest_load_dates = low_times.groupBy('date').count().orderBy('count').limit(3).collect()

# Display the results
print("The top 3 dates with the most anomalous demand times are: ")
for row in top_anomalous_dates:
    print(f"  - {row.date}")

print("The top 3 dates with the highest load are: ")
for row in top_highest_load_dates:
    print(f"  - {row.date}")

print("The top 3 dates with the lowest load are: ")
for row in top_lowest_load_dates:
    print(f"  - {row.date}")


The top 3 dates with the most anomalous demand times are: 
  - 2020-02-28
  - 2019-05-02
  - 2020-01-17
The top 3 dates with the highest load are: 
  - 2020-01-20
  - 2019-05-07
  - 2020-01-21
The top 3 dates with the lowest load are: 
  - 2016-05-17
  - 2018-03-17
  - 2016-04-23
